# ESP-AT test

Testprogramma voor de ESP-AT library.

* als er een mqtt-bericht ontvangen wordt, moet dit op het display getoond worden
* in de hoofdlus: rekening houden met wegvallen MQTT, WiFi.
* 

In [ ]:
from microbit import *
from espat import *
import radio

wifi_ssid = 'infvo-iot'
wifi_password = 'LISP77(Midas'
mqtt_user = 'mqtttest'
mqtt_password = 'testmqtt'
mqtt_host = 'infvopedia.nl'
mqtt_port = 1883

radio.on()

def log(msg: str):
    radio.send(msg)
    
def on_mqtt_message(topic: str, msg: str):
    log('mqtt-topic: ' + topic)
    log('mqtt-msg:' + msg)

# should this be done in the library???
uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
sleep(100)

display.show('0')
at_init_ESP()
display.show('1')
while not at_wifi_connected():
    try:
        at_wifi_connect(wifi_ssid, wifi_password)
    except Exception as exc:
        display.show(at_errorcode)
        sleep(2000)
        at_init_ESP()
        display.show('1')
display.show('2')

mac_address = at_get_mac_address()
log('My mac-addr: ' + mac_address)

at_mqtt_set_userconfig(1, 'gw-'+mac_address, mqtt_user, mqtt_password)
display.show('3')
    
while True:
    if not at_mqtt_connected():
        if not at_wifi_connected():
            reset()
        at_mqtt_connect(mqtt_host, mqtt_port, 1)
        at_set_on_mqtt_message(on_mqtt_message)
        at_mqtt_subscribe('microbit/input', 0)
        display.show('M')
    
    if button_a.was_pressed():
        if at_mqtt_connected():
            at_mqtt_publish('microbit/output', 'button A', 0, False)

    at_check_input()
    